In [11]:
# Packages
import os
import yfinance as yf
from haystack.nodes import LinkContentFetcher
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import  BM25Retriever, AnswerParser
from haystack.nodes.prompt import PromptTemplate
from haystack.nodes import PromptNode
from haystack import Pipeline

In [12]:
# Model configurations
hfAPIKey = os.getenv("HF_API_KEY")
hfModelName = os.getenv("HF_MODEL_NAME")

In [13]:
gold = yf.Ticker("GC=F")
news = gold.news

In [14]:
links = []

for new in news:
    links.append(new["link"])

In [15]:
lcf = LinkContentFetcher()
docs = []

for link in links:
    docs.append(lcf.fetch(link)[0])

In [16]:
documentStore = InMemoryDocumentStore(use_bm25=True)

retriever = BM25Retriever(document_store=documentStore,top_k=3)

documentStore.delete_documents()
documentStore.write_documents(docs)

Updating BM25 representation...: 100%|██████████| 8/8 [00:00<00:00, 3414.51 docs/s]


In [17]:
promptTemplate = PromptTemplate(prompt = 
                                """" You are a bot named GoldBot who helps people understand the commodity Gold. You share gold insights and investment strategies. Answer the following query based on the provided context in 500 characters max. If the context does
                                                not include an answer, reply with 'The data does not contain information related to the question'.\n
                                                Query: {query}\n
                                                Documents: {join(documents)}
                                                Answer: 
                                """,
                                output_parser=AnswerParser())

# Prompt Node initialization
promptNode = PromptNode(hfModelName,api_key=hfAPIKey,default_prompt_template=promptTemplate)

In [18]:
# Pipeline
pipeline = Pipeline()
pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipeline.add_node(component=promptNode, name="PromptNode", inputs=["Retriever"])

In [24]:
output = pipeline.run(query="Is it a good time to invest in Gold?",documents=documentStore.get_all_documents())
answer = output["answers"][0].answer
print(answer)

The prompt has been truncated from 6784 tokens to 924 tokens so that the prompt length and answer length (100 tokens) fit within the max token limit (1024 tokens). Shorten the prompt to prevent it from being cut off.


 which has been under pressure for a while. And we've been talking about that. But this week, it's been up. And it's been up for a few reasons. But one of the reasons is that inflation is still a concern. And that's something that we've been talking about for a while. And that's something that the Fed is still concerned about. And that's something that the market is still concerned about.
So that's one
